In [1]:
import phyloGraph as ph

from plotly.offline import download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)

#vim ~/.Plotly/.Credentials

In [2]:
import pandas as pd
import numpy as np

import re
import nltk

from gensim.models import word2vec, doc2vec

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#data_file = ("data/Chordata-2499-df.csv", 2499)
data_file = ("data/Mammalia-15040-df.csv", 15040)
#data_file = ("data/Primates-15963-df.csv", 15963)

## Load Data

In [4]:
pgd = ph.phyloData()

the data of phylo


Pick one of the methods below for getting data

### 1) fetch from Tree of Life

In [5]:
#pgd.search_name("primates")

In [6]:
#pgd.fetch_tol_data(15963, limit=25000)

In [7]:
#pgd.add_time()
#pgd.df.head()

### 2) load from raw file

In [8]:
#raw_file = "data/primate_raw.xml"
#pgd.load_raw_file(raw_file)

In [9]:
#pgd.write_prep_data("data/mammal_df.csv", "data/mammal_links.txt")

### 3) load from prepared files

In [10]:
#pgd.load_prep_data(*data_file)

### fix ages

In [11]:
#print(pgd.df.shape)
#pgd.df.head()

In [12]:
#pgd.fix_ages()

In [13]:
#pgd.write_prep_data(data_file[0].replace(".csv", "-fixedage.csv"))

In [14]:
pgd.load_prep_data(data_file[0].replace(".csv", "-fixedage.csv"), data_file[1])
print(pgd.df.shape)
pgd.df.head()

(1688, 12)


,ancestor,depth,extinct,id,name,num_kids,phylesis,x,y,Begin,End,log_time
0,14973,1,0,15040,Mammalia,2,0,0.985482,-0.609871,225.00,0.0,5.420535
1,15040,2,2,15989,Triconodonts,0,0,0.626042,-1.029422,190.00,70.0,5.252273
2,15040,2,0,15991,Monotremata,2,0,1.577066,-1.157830,210.00,0.0,5.351858
3,15991,3,0,16250,Tachyglossidae,4,0,1.934838,-1.035496,23.03,0.0,3.179303
4,16250,4,0,16251,Tachyglossus aculeatus,0,0,2.682316,-1.617251,11.50,0.0,2.525729


## Word2Vec

In [52]:
pgd.load_text_data(data_file[0].replace(".csv", "-text.csv"), cf_text=True, cf_level='species')
#pgd.text_df.head()
print(pgd.text_df.loc[pgd.text_df['id']==16421].squeeze()['text'])

chordata mammalia primates homo hominidae animalia h homo sapiens


In [53]:
def build_corpus(df):
    "Creates a list of lists containing words from each sentence"
    corpus = []
    for sentence in df['text'].iteritems():
        word_list = sentence[1].split(" ")
        corpus.append(word_list)
            
    return corpus

corpus = build_corpus(pgd.text_df)        
#corpus[0:2]

In [54]:
def build_docs(df):
    "Creates a list of lists containing words from each sentence"
    docs = []
    for i, row in df.iterrows():
        word_list = row['text'].split(" ")
        docs.append(doc2vec.TaggedDocument(word_list,[row['name']]))
            
    return docs

corpus = build_docs(pgd.text_df)        
corpus[0:2]

[TaggedDocument(words=['chordata', 'theriiformes', 'mammalialinnaeus', 'animalia', 'monotremata', 'yinotheria'], tags=['Mammalia']),
 TaggedDocument(words=['chordata', 'theriiformes', 'mammalialinnaeus', 'animalia', 'monotremata', 'yinotheria', 'triconodonts'], tags=['Triconodonts'])]

In [55]:
#?word2vec.Word2Vec

In [56]:
#model = word2vec.Word2Vec(corpus, size=100, window=3, min_count=1, workers=4)
#model.wv['primates']

In [210]:
?doc2vec.Doc2Vec

In [58]:
#?doc2vec.TaggedDocument

In [232]:
model = doc2vec.Doc2Vec(corpus, vector_size=100, window=5, min_count=5, workers=4, hs=0, negative=10, dm_mean=1)

In [233]:
#model.similar_by_word('primates')

In [234]:
#model.similar_by_word('sapiens')

In [235]:
#model.docvecs['Primates']

In [236]:
#?TSNE

In [237]:
def tsne_plot(model, df):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    #for word in model.wv.vocab:
    #    tokens.append(model[word])
    #    labels.append(word)
    
    for doc in list(df['name']):
        tokens.append(model.docvecs[doc])
        labels.append(doc)

    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [238]:
#tsne_plot(model, self.text_df)

In [239]:
def tsne_XY(model, df):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    #for word in model.wv.vocab:
    #    tokens.append(model[word])
    #    labels.append(word)
    
    for doc in list(df['name']):
        tokens.append(model.docvecs[doc])
        labels.append(doc)

    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    #new_values = tsne_model.fit_transform(tokens)
    new_values = TSNE().fit_transform(tokens)
    
    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
    
    return x, y

In [240]:
x, y = tsne_XY(model, pgd.text_df)

pgd.df['x'] = x
pgd.df['y'] = y

In [241]:
#pgd.cluster_XY(3, mode='pca')
#pgd.cluster_XY(3, mode='tsne')
#pgd.cluster_XY(3, mode='mds')
#pgd.df.head()

In [242]:
#pgd.df.tail()

## Plot Data

In [243]:
pgp = ph.phyloGraph(pgd)

the plotting of phylo


In [244]:
pgp.search_name('Mamma')

                 name     id
0            Mammalia  15040
1570  Mammalodontidae  16029


In [245]:
pgp.search_name('Homi')

          name     id
477  Hominidae  16299


In [246]:
#root_node=15040 # mammals
#focus_node = 16293 # old world monkeys

root_node=15997 # eutheria
focus_node = 16293 # old world monkeys

#root_node=15963 # primates
focus_node = 16299 # great apes
#focus_node = 16421 # humans
#focus_node = 'all'

In [247]:
pgp.create_plot_df(root_node)
#pgp.create_plot_df(root_node, max_depth=3)
#pgp.plot_df.head()

In [248]:
pgp.plot_df.shape

(1588, 12)

## plotting

In [249]:
pgp.create_plot_data()
#pgp.create_plot_data(root=root_node, focus=focus_node, add_links=True)
#pgp.create_plot_data(root=root_node, focus=focus_node, add_links=True, , Z_dim='depth', Z_dim_mult=-1)

Loaded plot data. Root node: 15997


In [250]:
#pgp.plot_df.to_csv("data/fake_plot_df_tsne.csv", index=False)
#pgp.plot_df = pd.read_csv("data/fake_plot_df_tsne.csv")

In [251]:
#pgp.plot_data
#pgp.render_plot()
#pgp.render_plot(publish=True, filename="Mammals")
#focus_node = 16300
pgp.refocus(focus_node)

/Users/Seth/Documents/tolGraph/phyloGraph.py:848: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Loaded plot data. Highlighting: 16299


In [252]:
#pgp.open_plot()